## This notebook will be used for the Coursera Capstone project 

In [9]:
import numpy as np
import pandas as pd

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library

import requests
import urllib.request
import time
from bs4 import BeautifulSoup
from urllib.request import urlopen

print('Libraries imported.')

Solving environment: ...working... done

# All requested packages already installed.

Solving environment: ...working... failed with initial frozen solve. Retrying with flexible solve.
Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\Jaime\Anaconda3

  added / updated specs:
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    altair-4.1.0               |             py_1         614 KB  conda-forge
    branca-0.4.1               |             py_0          26 KB  conda-forge
    certifi-2019.9.11          |           py37_0         147 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         8

In [10]:
print('Hello Capstone Project Course!')

Hello Capstone Project Course!


In [154]:
url = 'https://en.wikipedia.org/wiki/List_of_neighbourhoods_in_Toronto'
html = urlopen(url) 
soup = BeautifulSoup(html, 'html.parser')
tables = soup.find_all('table', {'class': "wikitable sortable"})
tables_list = []
for table in tables:
    tables_list.append(table)
    
table = tables_list[1]
table

<table class="wikitable sortable">
<tbody><tr bgcolor="lightblue">
<th width="5%">CDN number
</th>
<th width="20%">City-designated neighbourhood
</th>
<th width="10%">Former city/borough
</th>
<th width="50%">Neighbourhoods covered
</th>
<th width="15%">Map
</th></tr>
<tr>
<td>129
</td>
<td>Agincourt North
</td>
<td>Scarborough
</td>
<td><a href="/wiki/Agincourt,_Toronto" title="Agincourt, Toronto">Agincourt</a> and Brimwood
</td>
<td>
</td></tr>
<tr>
<td>128
</td>
<td>Agincourt South-Malvern West
</td>
<td>Scarborough
</td>
<td><a href="/wiki/Agincourt,_Toronto" title="Agincourt, Toronto">Agincourt</a> and <a href="/wiki/Malvern,_Toronto" title="Malvern, Toronto">Malvern</a>
</td>
<td><a class="image" href="/wiki/File:Malvern_to_locator.gif"><img alt="Malvern to locator.gif" data-file-height="149" data-file-width="291" decoding="async" height="51" src="//upload.wikimedia.org/wikipedia/commons/thumb/c/cd/Malvern_to_locator.gif/100px-Malvern_to_locator.gif" srcset="//upload.wikimedia.or

In [155]:
names = [] 
boroughs = []

tbodys = table.find_all('tbody')
for tbody in tbodys:
    rows = table.find_all('tr')
    for row in rows:
        cells = row.find_all('td')
        if len(cells) > 1:
            names.append(cells[1].text.strip())
            boroughs.append(cells[2].text.strip())
            

In [165]:
df = pd.DataFrame(names, columns = ['Neighborhood'])

In [166]:
df['Borough'] = boroughs
df['Latitude'] = None
df['Longitude'] = None
df = df.set_index('Neighborhood')

In [167]:
df

,Borough,Latitude,Longitude
Neighborhood,,,
Agincourt North,Scarborough,None,None
Agincourt South-Malvern West,Scarborough,None,None
Alderwood,Etobicoke,None,None
Annex,Old City of Toronto,None,None
Banbury-Don Mills,North York,None,None
Bathurst Manor,North York,None,None
Bay Street Corridor,Old City of Toronto,None,None
Bayview Village,North York,None,None
Bayview Woods-Steeles,North York,None,None


In [168]:
for name in df.index:   
    address = '{}, Toronto, ON'.format(name)
    print(address)

    geolocator = Nominatim(user_agent="toronto_explorer")
    location = geolocator.geocode(address)
    if location is None:
        continue
    latitude = location.latitude
    longitude = location.longitude
    df.loc[[name], ['Latitude']] = latitude
    df.loc[[name], ['Longitude']] = longitude
    print('The geograpical coordinates of {} are {}, {}.'.format(address, latitude, longitude))

Agincourt North, Toronto, ON
The geograpical coordinate of New York City are 43.8080383, -79.2664391.
Agincourt South-Malvern West, Toronto, ON
Alderwood, Toronto, ON
The geograpical coordinate of New York City are 43.6017173, -79.5452325.
Annex, Toronto, ON
The geograpical coordinate of New York City are 43.6703377, -79.407117.
Banbury-Don Mills, Toronto, ON
The geograpical coordinate of New York City are 43.7348039, -79.3572426.
Bathurst Manor, Toronto, ON
The geograpical coordinate of New York City are 43.6655189, -79.4119373.
Bay Street Corridor, Toronto, ON
The geograpical coordinate of New York City are 43.6673421, -79.3884571.
Bayview Village, Toronto, ON
The geograpical coordinate of New York City are 43.7691966, -79.3766617.
Bayview Woods-Steeles, Toronto, ON
The geograpical coordinate of New York City are 43.7981268, -79.3829726.
Bedford Park-Nortown, Toronto, ON
Beechborough-Greenbrook, Toronto, ON
Bendale, Toronto, ON
The geograpical coordinate of New York City are 43.75351

The geograpical coordinate of New York City are 43.6827255, -79.4380548.
Old East York, Toronto, ON
The geograpical coordinate of New York City are 43.699971000000005, -79.33251996261595.
Palmerston-Little Italy, Toronto, ON
The geograpical coordinate of New York City are 43.6600648, -79.4118056.
Parkwoods-Donalda, Toronto, ON
Pelmo Park-Humberlea, Toronto, ON
The geograpical coordinate of New York City are 43.7121949, -79.51755639837398.
Playter Estates-Danforth, Toronto, ON
Pleasant View, Toronto, ON
The geograpical coordinate of New York City are 43.787048, -79.3337137.
Princess-Rosethorn, Toronto, ON
Regent Park, Toronto, ON
The geograpical coordinate of New York City are 43.6607056, -79.3604569.
Rexdale-Kipling, Toronto, ON
The geograpical coordinate of New York City are 43.7213619, -79.5655129.
Rockcliffe-Smythe, Toronto, ON
Roncesvalles, Toronto, ON
The geograpical coordinate of New York City are 43.6514426, -79.4510381.
Rosedale-Moore Park, Toronto, ON
The geograpical coordinat

In [169]:
df

,Borough,Latitude,Longitude
Neighborhood,,,
Agincourt North,Scarborough,43.808,-79.2664
Agincourt South-Malvern West,Scarborough,None,None
Alderwood,Etobicoke,43.6017,-79.5452
Annex,Old City of Toronto,43.6703,-79.4071
Banbury-Don Mills,North York,43.7348,-79.3572
Bathurst Manor,North York,43.6655,-79.4119
Bay Street Corridor,Old City of Toronto,43.6673,-79.3885
Bayview Village,North York,43.7692,-79.3767
Bayview Woods-Steeles,North York,43.7981,-79.383


In [173]:
df1 = df.dropna()
df1

,Borough,Latitude,Longitude
Neighborhood,,,
Agincourt North,Scarborough,43.808,-79.2664
Alderwood,Etobicoke,43.6017,-79.5452
Annex,Old City of Toronto,43.6703,-79.4071
Banbury-Don Mills,North York,43.7348,-79.3572
Bathurst Manor,North York,43.6655,-79.4119
Bay Street Corridor,Old City of Toronto,43.6673,-79.3885
Bayview Village,North York,43.7692,-79.3767
Bayview Woods-Steeles,North York,43.7981,-79.383
Bendale,Scarborough,43.7535,-79.2553


In [178]:
# Getting address of Toronto
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Toronto is {}, {}.'.format(latitude, longitude))

The geograpical coordinates of Toronto is 43.6534817, -79.3839347.


In [181]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df1['Latitude'], df1['Longitude'], df1['Borough'], df1.index):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [185]:
toronto_data = df1[df1['Borough'] == 'Old City of Toronto']
toronto_data.head()

,Borough,Latitude,Longitude
Neighborhood,,,
Annex,Old City of Toronto,43.6703,-79.4071
Bay Street Corridor,Old City of Toronto,43.6673,-79.3885
Casa Loma,Old City of Toronto,43.6781,-79.4094
Church-Yonge Corridor,Old City of Toronto,43.6636,-79.3865
Corso Italia-Davenport,Old City of Toronto,43.678,-79.4431


In [186]:
address = 'Old City of Toronto, Toronto, ON'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Old City of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Old City of Toronto are 43.6204123, -79.373492.


In [187]:
# create map of Old Toronto using latitude and longitude values
map_old_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data.index):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_old_toronto)  
    
map_old_toronto

In [188]:
CLIENT_ID = 'LV3UMOYKQA3WRTV0VTAV0HMAAWNX2OZM20R2HIEVWSHIBDNM' # your Foursquare ID
CLIENT_SECRET = '53X5Y3AN4GTPN002MG0I525HOAVTSXM45MM3QMFLNKIDQ1JU' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: LV3UMOYKQA3WRTV0VTAV0HMAAWNX2OZM20R2HIEVWSHIBDNM
CLIENT_SECRET:53X5Y3AN4GTPN002MG0I525HOAVTSXM45MM3QMFLNKIDQ1JU


In [205]:
toronto_data

,Borough,Latitude,Longitude
Neighborhood,,,
Annex,Old City of Toronto,43.6703,-79.4071
Bay Street Corridor,Old City of Toronto,43.6673,-79.3885
Casa Loma,Old City of Toronto,43.6781,-79.4094
Church-Yonge Corridor,Old City of Toronto,43.6636,-79.3865
Corso Italia-Davenport,Old City of Toronto,43.678,-79.4431
Danforth Village - Toronto,Old City of Toronto,43.6834,-79.3236
Dufferin Grove,Old City of Toronto,43.6536,-79.4264
East End-Danforth,Old City of Toronto,43.6684,-79.3307
Forest Hill North,Old City of Toronto,43.6936,-79.4139


In [206]:
neighborhood_latitude = toronto_data.loc[toronto_data.index[0], 'Latitude'] # neighborhood latitude value
neighborhood_longitude = toronto_data.loc[toronto_data.index[0], 'Longitude'] # neighborhood longitude value

neighborhood_name = toronto_data.loc[toronto_data.index[0]] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Borough      Old City of Toronto
Latitude                 43.6703
Longitude               -79.4071
Name: Annex, dtype: object are 43.6703377, -79.407117.


In [207]:

LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL


'https://api.foursquare.com/v2/venues/explore?&client_id=LV3UMOYKQA3WRTV0VTAV0HMAAWNX2OZM20R2HIEVWSHIBDNM&client_secret=53X5Y3AN4GTPN002MG0I525HOAVTSXM45MM3QMFLNKIDQ1JU&v=20180605&ll=43.6703377,-79.407117&radius=500&limit=100'

In [208]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5f2b43289ed85e68a2c2f335'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'The Annex',
  'headerFullLocation': 'The Annex, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 42,
  'suggestedBounds': {'ne': {'lat': 43.6748377045, 'lng': -79.40090733739737},
   'sw': {'lat': 43.665837695499995, 'lng': -79.41332666260263}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4c0c36c0a1b32d7f97439cf0',
       'name': 'Jean Sibelius Square',
       'location': {'address': 'Wells St and Kendal Ave.',
        'lat': 43.67142628683921,
        'lng': -79.40883090368688,
        'labeledLatLngs': [{'label': 'display',
          'lat'

In [209]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [213]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Jean Sibelius Square,Park,43.671426,-79.408831
1,Roti Cuisine of India,Indian Restaurant,43.674618,-79.408249
2,Fresh on Bloor,Vegetarian / Vegan Restaurant,43.666755,-79.403491
3,The Original Gyro Grill,Greek Restaurant,43.666621,-79.405544
4,Fuwa Fuwa Japanese Pancakes,Pastry Shop,43.665880,-79.407840


In [214]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

42 venues were returned by Foursquare.


In [215]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [217]:
toronto_venues = getNearbyVenues(names=toronto_data.index,
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )



Annex
Bay Street Corridor
Casa Loma
Church-Yonge Corridor
Corso Italia-Davenport
Danforth Village - Toronto
Dufferin Grove
East End-Danforth
Forest Hill North
Forest Hill South
High Park North
High Park-Swansea
Junction Area
Kensington-Chinatown
Lawrence Park North
Lawrence Park South
Little Portugal
Moss Park
Mount Pleasant East
Mount Pleasant West
Niagara
North Riverdale
North St. James Town
Palmerston-Little Italy
Regent Park
Roncesvalles
Rosedale-Moore Park
Runnymede-Bloor West Village
South Riverdale
South Parkdale
The Beaches
Trinity-Bellwoods
University
Waterfront Communities-The Island
Wychwood
Yonge and Eglinton
Yonge-St.Clair


In [218]:
print(toronto_venues.shape)
toronto_venues.head()

(1366, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Annex,43.670338,-79.407117,Jean Sibelius Square,43.671426,-79.408831,Park
1,Annex,43.670338,-79.407117,Roti Cuisine of India,43.674618,-79.408249,Indian Restaurant
2,Annex,43.670338,-79.407117,Fresh on Bloor,43.666755,-79.403491,Vegetarian / Vegan Restaurant
3,Annex,43.670338,-79.407117,The Original Gyro Grill,43.666621,-79.405544,Greek Restaurant
4,Annex,43.670338,-79.407117,Fuwa Fuwa Japanese Pancakes,43.665880,-79.407840,Pastry Shop


In [219]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Annex,42,42,42,42,42,42
Bay Street Corridor,100,100,100,100,100,100
Casa Loma,23,23,23,23,23,23
Church-Yonge Corridor,32,32,32,32,32,32
Corso Italia-Davenport,22,22,22,22,22,22
Danforth Village - Toronto,33,33,33,33,33,33
Dufferin Grove,54,54,54,54,54,54
East End-Danforth,14,14,14,14,14,14
Forest Hill North,4,4,4,4,4,4


In [220]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 227 uniques categories.


In [221]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Accessories Store,Adult Boutique,Afghan Restaurant,American Restaurant,Animal Shelter,Antique Shop,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auto Dealership,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Castle,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,College Quad,College Theater,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Costume Shop,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Gas Station,Gastropub,German Restaurant,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hospital,Hostel,Hotel,IT Services,Ice Cream Shop,Indian Restaurant,Indie Theater,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kids Store,Korean Restaurant,Latin American Restaurant,Library,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Marijuana Dispensary,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Movie Theater,Museum,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,North Indian Restaurant,Office,Optical Shop,Organic Grocery,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Pet Store,Pharmacy,Pilates Studio,Pizza Place,Playground,Plaza,Polish Restaurant,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Residential Building (Apartment / Condo),Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Storage Facility,Supermarket,Sushi Restaurant,Syrian Restaurant,Taco Place,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Track,Trail,Tree,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Annex,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [222]:
toronto_onehot.shape

(1366, 227)

In [223]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Accessories Store,Adult Boutique,Afghan Restaurant,American Restaurant,Animal Shelter,Antique Shop,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auto Dealership,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Castle,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,College Quad,College Theater,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Costume Shop,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Gas Station,Gastropub,German Restaurant,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hospital,Hostel,Hotel,IT Services,Ice Cream Shop,Indian Restaurant,Indie Theater,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kids Store,Korean Restaurant,Latin American Restaurant,Library,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Marijuana Dispensary,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Movie Theater,Museum,Music Venue,Nail Salon,New American Restaurant,Nightclub,North Indian Restaurant,Office,Optical Shop,Organic Grocery,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Pet Store,Pharmacy,Pilates Studio,Pizza Place,Playground,Plaza,Polish Restaurant,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Residential Building (Apartment / Condo),Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Storage Facility,Supermarket,Sushi Restaurant,Syrian Restaurant,Taco Place,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Track,Trail,Tree,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wings Joint,Women's Store
0,Annex,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.023810,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.047619,0.023810,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.02381,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.071429,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.02381,0.000000,0.000000,0.000000,0.02381,0.000000,0.000000,0.000000,0.000000,0.02381,0.000000,0.00,0.023810,0.000000,0.000000,0.000000,0.000000,0.000000,0.023810,0.000000,0.000000,0.000000,0.000000,0.000

In [224]:
toronto_grouped.shape

(37, 227)

In [225]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Annex----
               venue  freq
0        Pizza Place  0.10
1        Coffee Shop  0.07
2     Ice Cream Shop  0.05
3  Indian Restaurant  0.05
4    Thai Restaurant  0.05


----Bay Street Corridor----
                 venue  freq
0          Coffee Shop  0.06
1       Clothing Store  0.05
2                 Café  0.05
3  Japanese Restaurant  0.05
4     Sushi Restaurant  0.04


----Casa Loma----
            venue  freq
0  Sandwich Place  0.13
1            Café  0.09
2  History Museum  0.09
3     Coffee Shop  0.09
4   Historic Site  0.04


----Church-Yonge Corridor----
              venue  freq
0       Coffee Shop  0.12
1             Diner  0.06
2       Yoga Studio  0.03
3  Sculpture Garden  0.03
4         Bookstore  0.03


----Corso Italia-Davenport----
                venue  freq
0  Italian Restaurant  0.23
1      Breakfast Spot  0.09
2         Coffee Shop  0.09
3    Sushi Restaurant  0.09
4     Thai Restaurant  0.05


----Danforth Village - Toronto----
                 venue  freq
0

In [226]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [227]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Annex,Pizza Place,Coffee Shop,Gym,Ice Cream Shop,Thai Restaurant,Bistro,Indian Restaurant,Sushi Restaurant,Ramen Restaurant,Falafel Restaurant
1,Bay Street Corridor,Coffee Shop,Clothing Store,Café,Japanese Restaurant,French Restaurant,Italian Restaurant,Sushi Restaurant,Boutique,Sandwich Place,Smoke Shop
2,Casa Loma,Sandwich Place,Café,Coffee Shop,History Museum,Pharmacy,Indian Restaurant,Castle,Middle Eastern Restaurant,Theater,Asian Restaurant
3,Church-Yonge Corridor,Coffee Shop,Diner,Yoga Studio,Ramen Restaurant,Beer Bar,Japanese Restaurant,Bookstore,Sculpture Garden,Bubble Tea Shop,Salon / Barbershop
4,Corso Italia-Davenport,Italian Restaurant,Breakfast Spot,Sushi Restaurant,Coffee Shop,Mediterranean Restaurant,Brazilian Restaurant,Thai Restaurant,Mexican Restaurant,Lounge,Pet Store


In [228]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 1, 1, 1, 1, 0, 1, 4, 4])

In [229]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,Borough,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
Neighborhood,,,,,,,,,,,,,,
Annex,Old City of Toronto,43.6703,-79.4071,1,Pizza Place,Coffee Shop,Gym,Ice Cream Shop,Thai Restaurant,Bistro,Indian Restaurant,Sushi Restaurant,Ramen Restaurant,Falafel Restaurant
Bay Street Corridor,Old City of Toronto,43.6673,-79.3885,1,Coffee Shop,Clothing Store,Café,Japanese Restaurant,French Restaurant,Italian Restaurant,Sushi Restaurant,Boutique,Sandwich Place,Smoke Shop
Casa Loma,Old City of Toronto,43.6781,-79.4094,1,Sandwich Place,Café,Coffee Shop,History Museum,Pharmacy,Indian Restaurant,Castle,Middle Eastern Restaurant,Theater,Asian Restaurant
Church-Yonge Corridor,Old City of Toronto,43.6636,-79.3865,1,Coffee Shop,Diner,Yoga Studio,Ramen Restaurant,Beer Bar,Japanese Restaurant,Bookstore,Sculpture Garden,Bubble Tea Shop,Salon / Barbershop
Corso Italia-Davenport,Old City of Toronto,43.678,-79.4431,1,Italian Restaurant,Breakfast Spot,Sushi Restaurant,Coffee Shop,Mediterranean Restaurant,Brazilian Restaurant,Thai Restaurant,Mexican Restaurant,Lounge,Pet Store


In [231]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged.index, toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [232]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Latitude,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
Neighborhood,,,,,,,,,,
Dufferin Grove,43.6536,Coffee Shop,Restaurant,Italian Restaurant,Breakfast Spot,Sports Bar,Bakery,Portuguese Restaurant,Mexican Restaurant,Convenience Store
High Park North,43.6574,Convenience Store,Mattress Store,Baseball Field,Gym / Fitness Center,Concert Hall,Comic Shop,Dumpling Restaurant,Drugstore,Donut Shop
Junction Area,43.6655,Italian Restaurant,Café,Bar,Thai Restaurant,Deli / Bodega,Antique Shop,Nail Salon,Indian Restaurant,Grocery Store
Kensington-Chinatown,43.6534,Vegetarian / Vegan Restaurant,Coffee Shop,Vietnamese Restaurant,Dessert Shop,Bar,Mexican Restaurant,Burger Joint,Gaming Cafe,Pizza Place
Little Portugal,43.6474,Café,Coffee Shop,Bakery,Restaurant,Cocktail Bar,Korean Restaurant,Vegetarian / Vegan Restaurant,Park,Japanese Restaurant
Niagara,43.6441,Bakery,Pizza Place,Dessert Shop,Spa,Café,Middle Eastern Restaurant,Park,Paper / Office Supplies Store,Optical Shop
North Riverdale,43.6655,Chinese Restaurant,Bakery,Coffee Shop,Fast Food Restaurant,Light Rail Station,Bar,Asian Restaurant,French Restaurant,Ice Cream Shop
Palmerston-Little Italy,43.6601,Taco Place,Asian Restaurant,Salon / Barbershop,Juice Bar,Dessert Shop,Indie Theater,Fish & Chips Shop,Pizza Place,Tapas Restaurant
Roncesvalles,43.6514,Gift Shop,Gourmet Shop,Restaurant,Breakfast Spot,Bookstore,Italian Restaurant,Pub,Dessert Shop,Sports Bar


In [233]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Latitude,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
Neighborhood,,,,,,,,,,
Annex,43.6703,Coffee Shop,Gym,Ice Cream Shop,Thai Restaurant,Bistro,Indian Restaurant,Sushi Restaurant,Ramen Restaurant,Falafel Restaurant
Bay Street Corridor,43.6673,Clothing Store,Café,Japanese Restaurant,French Restaurant,Italian Restaurant,Sushi Restaurant,Boutique,Sandwich Place,Smoke Shop
Casa Loma,43.6781,Café,Coffee Shop,History Museum,Pharmacy,Indian Restaurant,Castle,Middle Eastern Restaurant,Theater,Asian Restaurant
Church-Yonge Corridor,43.6636,Diner,Yoga Studio,Ramen Restaurant,Beer Bar,Japanese Restaurant,Bookstore,Sculpture Garden,Bubble Tea Shop,Salon / Barbershop
Corso Italia-Davenport,43.678,Breakfast Spot,Sushi Restaurant,Coffee Shop,Mediterranean Restaurant,Brazilian Restaurant,Thai Restaurant,Mexican Restaurant,Lounge,Pet Store
Danforth Village - Toronto,43.6834,Coffee Shop,Grocery Store,Gastropub,American Restaurant,Bar,Ethiopian Restaurant,Burger Joint,Bank,Pizza Place
East End-Danforth,43.6684,Burger Joint,Indie Theater,Mexican Restaurant,Skating Rink,Polish Restaurant,Thrift / Vintage Store,Café,Park,Egyptian Restaurant
Lawrence Park North,43.7292,Coffee Shop,Bakery,Italian Restaurant,Fast Food Restaurant,Pub,Cosmetics Shop,Bank,Pharmacy,Asian Restaurant
Lawrence Park South,43.7292,Coffee Shop,Bakery,Italian Restaurant,Fast Food Restaurant,Pub,Cosmetics Shop,Bank,Pharmacy,Asian Restaurant


In [234]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Latitude,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
Neighborhood,,,,,,,,,,
High Park-Swansea,43.6455,Bus Line,Bar,Dance Studio,Dessert Shop,Dumpling Restaurant,Drugstore,Donut Shop,Doner Restaurant,Dog Run


In [235]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Latitude,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
Neighborhood,,,,,,,,,,
Rosedale-Moore Park,43.6904,Trail,Tennis Court,Park,Gift Shop,Dance Studio,Donut Shop,Doner Restaurant,Dog Run,Dive Bar


In [236]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Latitude,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
Neighborhood,,,,,,,,,,
Forest Hill North,43.6936,Playground,Mediterranean Restaurant,Bank,Design Studio,Drugstore,Donut Shop,Doner Restaurant,Dog Run,Dive Bar
Forest Hill South,43.6936,Playground,Mediterranean Restaurant,Bank,Design Studio,Drugstore,Donut Shop,Doner Restaurant,Dog Run,Dive Bar
